<center>

## Term Project: Analysing Trends in Movie Performance and Ratings
### Milestone 5: Merging the Data and Storing in a Database/Visualizing Data
### Karthika Vellingiri
### 15 Nov 2024
</center>

### Milestone 2: Cleaning/Formatting Flat File Source

In [ ]:
# Import necessary libraries
import pandas as pd
import numpy as np
import ast
import re
from fuzzywuzzy import fuzz
import langcodes  # Import the langcodes library to handle language codes
import warnings
from IPython.display import display, HTML

# Suppress warnings
warnings.filterwarnings("ignore", category=DeprecationWarning)

# Step #1: Load the dataset
df = pd.read_csv("movies_metadata.csv", low_memory=False)  # Load the movie metadata CSV file
print(f"Data file 'movies_metadata.csv' loaded successfully.\n")

print("Starting Data cleanup...")  # Indicate that data cleanup is starting

# Step #2: Drop specified columns that are not needed for analysis
columns_to_drop = ['adult', 'belongs_to_collection', 'homepage', 'overview', 'poster_path', 'tagline', 'video']
df = df.drop(columns=columns_to_drop, errors='ignore')  # Drop columns, ignore errors if columns are missing

# Step #3: Remove duplicate rows from the dataset
initial_row_count = df.shape[0]  # Store initial row count for reference
df = df.drop_duplicates()  # Remove duplicate entries

# Step #4: Remove rows with specific criteria that do not meet quality standards
df = df[
    (df['genres'].apply(lambda x: bool(ast.literal_eval(x)) if pd.notnull(x) else False)) &  # Check if genres are valid
    (df['production_companies'].apply(lambda x: bool(ast.literal_eval(x)) if pd.notnull(x) else False)) &  # Check production companies
    (df['production_countries'].apply(lambda x: bool(ast.literal_eval(x)) if pd.notnull(x) else False)) &  # Check production countries
    (df['spoken_languages'].apply(lambda x: bool(ast.literal_eval(x)) if pd.notnull(x) else False)) &  # Check spoken languages
    (df['imdb_id'].notna()) &  # Ensure IMDb ID is present
    (df['imdb_id'] != '') &  # Ensure IMDb ID is not empty
    (df['original_language'].notna()) &  # Ensure original language is present
    (df['original_language'] != '') &  # Ensure original language is not empty
    (df['revenue'].notna()) &  # Ensure revenue is present
    (df['revenue'] != 0) &  # Ensure revenue is greater than zero
    (df['popularity'] != 0)  # Ensure popularity is greater than zero
]

# Step #5: Filter 'original_title' and 'title' to remove entries with junk values and numbers
#df = df[~df['original_title'].str.contains(r'[^\x00-\x7F]|[0-9]', regex=True)]  # Remove titles with non-ASCII characters or numbers
df = df[~(df['original_title'].str.contains(r'[0-9:&a-z]', regex=True) | df['title'].str.contains(r'[0-9:&a-z]', regex=True))]


# Step #6: Validate 'production_companies' for proper names and IDs
df = df[df['production_companies'].apply(lambda x: all(
    isinstance(company.get('name', ''), str) and company.get('name') and isinstance(company.get('id', 0), int)
    for company in ast.literal_eval(x)) if pd.notnull(x) else False)]  # Ensure each company has a valid name and ID

# Step #7: Validate 'production_countries' for proper ISO codes and names
df = df[df['production_countries'].apply(lambda x: all(
    isinstance(country.get('iso_3166_1', ''), str) and len(country.get('iso_3166_1')) == 2 and
    isinstance(country.get('name', ''), str) and country.get('name')
    for country in ast.literal_eval(x)) if pd.notnull(x) else False)]  # Ensure each country has valid ISO codes and names

# Step #8: Remove rows with inappropriate revenue values (less than 10,000)
df = df[df['revenue'] > 10000]  # Filter out low-revenue entries

# Step #9: Remove rows with inappropriate budget values (less than 50,000)
df['budget'] = pd.to_numeric(df['budget'], errors='coerce')  # Convert budget to numeric, coerce errors to NaN
df = df[df['budget'] > 50000]  # Filter out low-budget entries

# Step #10: Handle 'runtime' column for missing values
avg_runtime = 90  # Define average runtime to use for filtering
df['runtime'] = df['runtime'].apply(lambda x: avg_runtime if pd.isnull(x) else x)  # Fill missing runtime with average value

# Remove rows with runtime less than 90 minutes and greater than 180 minutes
df = df[(df['runtime'] >= 90) & (df['runtime'] <= 180)]  # Filter for valid runtime entries

# Step #11: Derive 'genres' from the dataset
def derive_genres(genres):
    """Extracts genre names from the JSON-like structure in the 'genres' column."""
    if pd.isnull(genres):  # Check for null values
        return ""
    try:
        genre_list = ast.literal_eval(genres)  # Convert string to list
        return ", ".join([genre['name'] for genre in genre_list])  # Return a comma-separated string of genre names
    except (ValueError, SyntaxError):  # Handle potential parsing errors
        return ""

df['genres'] = df['genres'].apply(derive_genres)  # Apply the genre extraction function

# Step #12: Create multigenre column to indicate if multiple genres are present
def check_multigenre(genres):
    """Check if a movie has multiple genres."""
    if pd.isnull(genres) or genres == "":  # Check for null or empty genres
        return "no"  # Return "no" if no genres are present
    genre_list = [genre.strip() for genre in genres.split(",")]  # Split genres into a list
    return "yes" if len(genre_list) > 1 else "no"  # Return "yes" if multiple genres exist

df['multigenre'] = df['genres'].apply(check_multigenre)  # Apply the multigenre check

# Step #13: Process spoken languages to create multilanguage column
def process_spoken_languages(languages):
    """Convert spoken languages data into a comma-separated string and check if multiple languages exist."""
    if pd.isnull(languages):  # Check for null languages
        return "", "no"  # Return empty string and "no"
    
    language_list = [language['iso_639_1'] for language in ast.literal_eval(languages)]  # Extract ISO language codes
    
    # Use langcodes to map language codes to names
    mapped_languages = [langcodes.Language.get(lang_code).language_name() for lang_code in language_list]
    
    # Filter out any that didn't map successfully
    cleaned_languages = [lang for lang in mapped_languages if lang]  # Keep only successfully mapped languages
    
    multilanguage = "yes" if len(cleaned_languages) > 1 else "no"  # Check if there are multiple languages
    return ", ".join(cleaned_languages), multilanguage  # Return cleaned languages and multilanguage indicator

# Apply the process_spoken_languages function to the spoken_languages column
df[['spoken_languages', 'multilanguage']] = df['spoken_languages'].apply(lambda x: pd.Series(process_spoken_languages(x)))

# Step #14: Process production companies to create multicompany column
def process_production_companies(companies):
    """Convert production companies data into a comma-separated string and check if multiple companies exist."""
    if pd.isnull(companies):  # Check for null companies
        return "", "no"  # Return empty string and "no"
    
    company_list = [company['name'] for company in ast.literal_eval(companies)]  # Extract company names
    multicompany = "yes" if len(company_list) > 1 else "no"  # Check if there are multiple companies
    return ", ".join(company_list), multicompany  # Return company names and multicompany indicator

# Apply the process_production_companies function to the production_companies column
df[['production_companies', 'multicompany']] = df['production_companies'].apply(lambda x: pd.Series(process_production_companies(x)))

# Step #15: Process production countries to create multicountry column
def process_production_countries(countries):
    """Convert production countries data into a comma-separated string and check if multiple countries exist."""
    if pd.isnull(countries):  # Check for null countries
        return "", "no"  # Return empty string and "no"
    
    country_list = [country['name'] for country in ast.literal_eval(countries)]  # Extract country names
    multicountry = "yes" if len(country_list) > 1 else "no"  # Check if there are multiple countries
    return ", ".join(country_list), multicountry  # Return country names and multicountry indicator

# Apply the process_production_countries function to the production_countries column
df[['production_countries', 'multicountry']] = df['production_countries'].apply(lambda x: pd.Series(process_production_countries(x)))

# Step #16: Fuzzy match 'original_title' and 'title' to ensure they are similar
df = df[df.apply(lambda row: fuzz.ratio(row['original_title'], row['title']) > 80, axis=1)]  # Keep only rows with high similarity

# Step #17: Filter 'status' column for only "Released" status
df = df[df['status'] == 'Released']  # Retain only movies that have been released

# Step #18: Filter 'vote_average' and 'vote_count' columns for appropriate values
df = df[~((df['vote_average'] < 5.0) & (df['vote_count'] < 100))]  # Exclude low-rated and low-vote movies

# Step #19: Validate Data Types
def validate_data_types(df):
    """Check if numeric columns are indeed numeric and categorical columns are strings."""
    expected_types = {
        'budget': 'numeric',
        'revenue': 'numeric',
        'runtime': 'numeric',
        'vote_average': 'numeric',
        'vote_count': 'numeric',
        'original_language': 'string',
        'status': 'string',
        'imdb_id': 'string'
    }
    
    for column, expected_type in expected_types.items():
        if expected_type == 'numeric' and not pd.api.types.is_numeric_dtype(df[column]):
            print(f"Warning: {column} is not numeric!")  # Alert if column is not numeric
        elif expected_type == 'string' and not pd.api.types.is_string_dtype(df[column]):
            print(f"Warning: {column} is not a string!")  # Alert if column is not string

validate_data_types(df)  # Validate the data types of the DataFrame

# Step #20: Convert 'popularity' to numeric, forcing errors to NaN
df['popularity'] = pd.to_numeric(df['popularity'], errors='coerce')  # Convert popularity to numeric

# Step #21: Round off popularity to 2 decimal points
df['popularity'] = df['popularity'].round(2)  # Round popularity to 2 decimal points

# Step #22: Format revenue and budget in dollars
df['revenue'] = df['revenue'].apply(lambda x: f"${x:,.2f}")  # Format revenue with dollar sign and commas
df['budget'] = df['budget'].apply(lambda x: f"${x:,.2f}")  # Format budget with dollar sign and commas

# Step #23: Remove decimal from vote count
df['vote_count'] = df['vote_count'].astype(int)  # Convert vote count to integer to remove decimals

# Step #24: Convert column names to lowercase for consistency
df.columns = [col.lower() for col in df.columns]  # Change all column names to lowercase

# Indicate the completion of data cleanup
print("Data cleanup completed successfully!!!")  # Notify that the cleanup process has finished

# Step #25: Rename runtime column and remove decimal
df.rename(columns={'runtime': 'runtime(in_minutes)'}, inplace=True)  # Rename runtime column
df['runtime(in_minutes)'] = df['runtime(in_minutes)'].astype(int)  # Convert to integer to remove decimals

# Step #26: Print dataset before and after cleanup for comparison
display(HTML("<br><center><h3><strong>Dataset before cleanup</h3></strong></center>"))

initial_df = pd.read_csv("movies_metadata.csv", low_memory=False)  # Load the original dataset again for comparison
display(HTML(initial_df.head().to_html(index=False)))  # Display the first few rows of the original dataset

display(HTML("<br><center><strong><h3>Dataset after cleanup</h3></strong></center>"))
display(HTML(df.head().to_html(index=False)))  # Display the first few rows of the cleaned dataset

# Save the cleaned dataset to a new CSV file
cleaned_file = "cleaned_movies_metadata.csv"  # Define the output file path
df.to_csv(cleaned_file, index=False)  # Save the cleaned DataFrame to CSV
print(f"Cleaned dataset saved to {cleaned_file}.\n")  # Notify that the cleaned dataset has been saved


### Milestone 3: Cleaning/Formatting Website Data

In [ ]:
from bs4 import BeautifulSoup  # Import BeautifulSoup for HTML parsing
import pandas as pd  # Import pandas for data manipulation
import requests  # Import requests to make HTTP requests
from IPython.display import display, HTML  # Import display functions for HTML output

# Function to extract data for a given year
def extract_data(year):
    url = f"https://www.boxofficemojo.com/year/world/{year}/"  # Construct URL with the year
    response = requests.get(url)  # Send a request to the URL
    soup = BeautifulSoup(response.text, "html.parser")  # Parse the HTML content
    
    # Find the table with movie data on the page
    table = soup.find("table")
    if table is None:  # If no table is found, return an empty DataFrame
        return pd.DataFrame()
    
    rows = table.find_all("tr")  # Find all rows in the table
    
    # Extract column headers from the first row (header row)
    headers = [header.text.strip() for header in rows[0].find_all("th")]
    
    # Extract data from each row after the header row
    data = []
    for row in rows[1:]:
        cols = row.find_all("td")  # Find all columns in each row
        cols = [col.text.strip() for col in cols]  # Strip whitespace from each cell
        data.append(cols)  # Append row data to the data list
    
    # Create a DataFrame from the extracted data and add 'Year' column
    df = pd.DataFrame(data, columns=headers)
    df['Year'] = year  # Add the year to the DataFrame
    return df  # Return the DataFrame for the year

# Step 1: Loop through years and extract data
print("Starting Data Extraction . . .")  # Initial message
all_data = []  # Initialize list to store all years' data
for i, year in enumerate(range(1977, 2021), start=1):  # Loop through years 1977 to 2020
    year_data = extract_data(year)  # Call function to extract data for the year
    all_data.append(year_data)  # Append the DataFrame to all_data
    if year != 2020:
        print(f"Data Extraction completed for the year {year}...", end='\r')  # Print message for each year, replacing in-place
    else:
        print("Data Extraction completed Successfully ! ! ! ", flush=True)  # Print completion message for the last year

# Step 2: Combine all data into a single DataFrame
combined_df = pd.concat(all_data, ignore_index=True)  # Concatenate all DataFrames into one large DataFrame
# Copy combined_df to original_df, excluding the 'Year' column to have the exact table content
original_df = combined_df.drop(columns=['Year']).copy()  # Drop 'Year' and create a copy for comparison

# Step 3: Sort combined DataFrame by 'Worldwide' before cleaning
combined_df['Worldwide'] = combined_df['Worldwide'].replace({'\$': '', ',': ''}, regex=True).astype(float)  # Remove dollar signs and commas, convert to float for sorting
combined_df = combined_df.sort_values(by='Worldwide', ascending=False).reset_index(drop=True)  # Sort by Worldwide revenue

# Step 4: Trim whitespace from all cells
combined_df = combined_df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)  # Strip whitespace from string columns

# Step 5: Replace empty strings with NaN values
combined_df = combined_df.replace('', pd.NA)  # Replace any empty strings with NaN

# Step 6: Convert 'Worldwide' and 'Domestic' columns to numeric by removing currency symbols and commas
combined_df['Worldwide_numeric'] = combined_df['Worldwide'].replace({'\$': '', ',': '', '-': '0'}, regex=True).astype(float)  # Remove symbols, commas, convert to float
combined_df['Domestic_numeric'] = combined_df['Domestic'].replace({'\$': '', ',': '', '-': '0'}, regex=True).astype(float)  # Same for Domestic column

# Step 7: Derive 'Foreign' and 'Foreign %' columns
combined_df['Foreign'] = combined_df['Worldwide_numeric'] - combined_df['Domestic_numeric']  # Calculate Foreign revenue as Worldwide - Domestic
combined_df['Foreign %'] = round((combined_df['Foreign'] / combined_df['Worldwide_numeric']) * 100, 2)  # Calculate Foreign percentage

# Step 8: Derive 'Domestic %' column
combined_df['Domestic %'] = round((combined_df['Domestic_numeric'] / combined_df['Worldwide_numeric']) * 100, 2)  # Calculate Domestic percentage

# Step 9: Format the 'Domestic %' column to display as a percentage
combined_df['Domestic %'] = combined_df['Domestic %'].astype(str) + '%'  # Append '%' symbol to Domestic percentage

# Step 10: Format monetary columns to display as currency
combined_df['Worldwide'] = combined_df['Worldwide_numeric'].apply(lambda x: f"${round(x):,d}" if pd.notnull(x) else x)  # Format Worldwide as currency
combined_df['Domestic'] = combined_df['Domestic_numeric'].apply(lambda x: f"${round(x):,d}" if pd.notnull(x) else x)  # Format Domestic as currency
combined_df['Foreign'] = combined_df['Foreign'].apply(lambda x: f"${round(x):,d}" if pd.notnull(x) else x)  # Format Foreign as currency

# Step 11: Remove records where 'Domestic_numeric' is 0
combined_df = combined_df[combined_df['Domestic_numeric'] != 0]  # Filter out records with zero Domestic revenue

# Step 12: Drop the 'Rank' column if it exists
if 'Rank' in combined_df.columns:
    combined_df = combined_df.drop(columns=['Rank'])  # Drop Rank column if present

# Step 13: Remove duplicates based on 'Release Group'
# Convert 'Release Group' to lowercase for case-insensitive duplicate removal
combined_df['Release Group_lower'] = combined_df['Release Group'].str.lower()  # Convert all Release Group entries to lowercase

# Remove duplicate entries based on 'Release Group', keeping the first occurrence
combined_df = combined_df.drop_duplicates(subset=['Release Group_lower'], keep='first')  

# Step 14: Rank the data by Worldwide revenue
combined_df['Rank'] = combined_df['Worldwide_numeric'].rank(method='min', ascending=False).astype(int)  # Rank movies by Worldwide revenue
combined_df = combined_df.sort_values(by='Rank').reset_index(drop=True)  # Sort the DataFrame by Rank

# Step 15: Remove records where 'Worldwide' is not equal to 'Domestic' and 'Foreign %' is 0
combined_df = combined_df[~((combined_df['Domestic_numeric'] == combined_df['Worldwide_numeric']) & (combined_df['Foreign %'] == 0))]  # Remove movies where Worldwide == Domestic and Foreign % is 0

# Step 16: Drop the temporary numeric columns
combined_df = combined_df.drop(columns=['Worldwide_numeric', 'Domestic_numeric', 'Release Group_lower'])  # Drop numeric columns used for calculations

# Step 17: Drop the '%' column, as it is replaced by new columns
combined_df = combined_df.drop(columns=['%'], errors='ignore')  # Drop '%' column if it exists

# Step 18: Select relevant columns for final output
columns_to_print = ['Rank', 'Release Group', 'Worldwide', 'Domestic', 'Domestic %', 'Foreign', 'Foreign %', 'Year']  # Specify the columns to display
combined_df = combined_df[columns_to_print]  # Filter the DataFrame to keep only the selected columns

# Print original data before cleaning
display(HTML("<br><strong><h3>Dataset before cleanup:</h3></br></strong>"))  # Display heading for original data
display(HTML(original_df.head(10).to_html(index=False)))  # Display first 10 rows of the original DataFrame

# Step 19: Display the cleaned dataset
# Convert DataFrame to HTML
html_table = combined_df.head(10).to_html(index=False)  # Convert first 10 rows of the cleaned DataFrame to HTML

# Align all headers to the center
html_table = html_table.replace('<th>', '<th style="text-align: center;">')  # Align headers to center

# Align all data cells to the left
html_table = html_table.replace('<td>', '<td style="text-align: left;">')  # Align data cells to the left

display(HTML("<br><strong><h3>Dataset after cleanup:</h3></br></strong>"))  # Display heading for cleaned data

# Display the modified HTML table
display(HTML(html_table))  # Display the cleaned DataFrame

# Step 20: Save cleaned data to CSV file
combined_df.to_csv("cleaned_box_office_data.csv", index=False)  # Save the cleaned DataFrame to a CSV file


### Milestone 4: Connecting to an API/Pulling in the Data and Cleaning/Formatting

In [ ]:
import requests  # Importing requests library for making HTTP requests to APIs
from tabulate import tabulate  # Importing tabulate for creating nicely formatted tables
from IPython.display import display, HTML  # Importing display and HTML for showing HTML content in Jupyter notebooks
import re  # Importing regular expressions for string matching and extraction
import csv  # Importing csv for handling CSV file operations
import json  # Importing json to read the API key from a JSON file

# Step 1: Read the OMDB API key from the JSON file
with open('OMDbAPIkey.json') as f:  # Open the JSON file containing the API key
    api_key_data = json.load(f)  # Load the JSON data
    api_key = api_key_data["OMDb_API_Key"]  # Extract the API key

# Step 2: Function to fetch movies for a specific year from the OMDB API
def fetch_movies_for_year(year):
    print(f"Fetching movies for the year {year}", end='\r')  # Indicate progress in the console
    url = f"http://www.omdbapi.com/?apikey={api_key}&y={year}&s=movie"  # Construct the API URL
    response = requests.get(url)  # Make the GET request to the OMDB API
    
    if response.status_code == 200:  # Check if the response is successful
        data = response.json()  # Parse the JSON data from the response
        results = data.get("Search", [])  # Get the list of movies found, defaulting to an empty list if none
        return results  # Return the list of movies
    else:
        print(f"Failed to fetch movies for {year}. Status code: {response.status_code}")  # Print error if the request failed
        return []  # Return an empty list if the fetch failed

# Step 3: Function to fetch detailed movie data using its IMDB ID
def fetch_movie_details(imdb_id):
    url = f"http://www.omdbapi.com/?apikey={api_key}&i={imdb_id}"  # Construct the URL for fetching details
    response = requests.get(url)  # Make the GET request for detailed movie data
    if response.status_code == 200:  # Check if the response is successful
        return response.json()  # Return the parsed JSON data
    else:
        print(f"Failed to fetch details for IMDB ID: {imdb_id}. Status code: {response.status_code}")  # Print error if the request failed
        return None  # Return None if the fetch failed

# Step 4: Function to extract wins and nominations from the awards string
def extract_awards(awards):
    wins, nominations = 0, 0  # Initialize wins and nominations to 0
    # Match the pattern "X wins" in the awards string
    wins_match = re.search(r'(\d+)\s+wins?', awards, re.IGNORECASE)
    # Match the pattern "X nominations" in the awards string
    nominations_match = re.search(r'(\d+)\s+nominations?', awards, re.IGNORECASE)
    
    if wins_match:  # If wins are found in the awards string
        wins = int(wins_match.group(1))  # Convert the captured group to an integer
    if nominations_match:  # If nominations are found in the awards string
        nominations = int(nominations_match.group(1))  # Convert the captured group to an integer
    
    return wins, nominations  # Return the extracted wins and nominations

# Step 5: List to hold all movies data
all_movies_data = []  # Initialize an empty list to store all movie data

# Step 6: Specify the years for data extraction 
years_to_fetch = list(range(1978, 2024))  # Create a list of years from 1978 to 2023

print(f"Data Extraction Initiated...")  # Print message indicating data extraction has started

# Step 7: Loop through each specified year to fetch movie data
for idx, year in enumerate(years_to_fetch):  # Enumerate over the years for indexing
    movies = fetch_movies_for_year(year)  # Fetch movies for the current year
    
    for movie in movies:  # Loop through the list of fetched movies
        title = movie.get("Title")  # Get the movie title
        year = movie.get("Year")  # Get the year of the movie
        
        # Check if the year contains a hyphen and extract the last part
        if '-' in year:
            year = year.split('-')[-1].strip()  # Split and take the last part
        elif '–' in year:  # Handle the en-dash character
            year = year.split('–')[-1].strip()  # Split and take the last part
            
        imdb_id = movie.get("imdbID")  # Get the IMDB ID for detailed fetching
        movie_type = movie.get("Type")  # Get the type of the movie (e.g., movie, series)
        
        # Step 8: Only process if the movie type is 'movie'
        if movie_type == "movie":
            # Step 9: Fetch detailed data for each movie to get more info
            detailed_movie_data = fetch_movie_details(imdb_id)  # Get detailed movie data
            if detailed_movie_data:  # If detailed data is fetched successfully
                # Step 10: Extract awards, wins, and nominations
                awards = detailed_movie_data.get("Awards", "")  # Get the awards string
                wins, nominations = extract_awards(awards)  # Extract wins and nominations from awards
                
                # Step 11: Extract additional movie attributes
                genres = detailed_movie_data.get("Genre", "").split(", ")  # Get genres and split into a list
                languages = detailed_movie_data.get("Language", "").split(", ")  # Get languages and split into a list
                countries = detailed_movie_data.get("Country", "").split(", ")  # Get countries and split into a list
                productions = detailed_movie_data.get("Production", "").split(", ")  # Get production companies and split into a list

                # Step 12: Additional fields
                released = detailed_movie_data.get("Released", "N/A")  # Get release date, default to "N/A"
                runtime = detailed_movie_data.get("Runtime", "N/A")  # Get runtime, default to "N/A"
                
                # Remove 'min' from runtime and convert to a usable format
                if isinstance(runtime, str) and 'min' in runtime:
                    runtime = runtime.replace(' min', '').strip()  # Clean up the runtime string
                else:
                    runtime = "N/A"  # Default to "N/A" if not a valid string
                
                director = detailed_movie_data.get("Director", "N/A")  # Get director name
                metascore = detailed_movie_data.get("Metascore", "N/A")  # Get metascore value
                imdb_rating = detailed_movie_data.get("imdbRating", "N/A")  # Get IMDB rating
                imdb_votes = detailed_movie_data.get("imdbVotes", "N/A")  # Get number of IMDB votes
                box_office = detailed_movie_data.get("BoxOffice", "N/A")  # Get box office collection

                # Step 13: Determine if the movie has multiple genres, languages, companies, or countries
                multigenre = "Yes" if len(genres) > 1 else "No"  # Check for multiple genres
                multilanguage = "Yes" if len(languages) > 1 else "No"  # Check for multiple languages
                multicompany = "Yes" if len(productions) > 1 else "No"  # Check for multiple production companies
                multicountry = "Yes" if len(countries) > 1 else "No"  # Check for multiple countries

                # Step 14: Prepare a dictionary of all relevant movie data
                ratings_dict = {rating['Source']: rating['Value'].split('/')[0] for rating in detailed_movie_data.get("Ratings", [])}
                imdb_rating_value = ratings_dict.get("Internet Movie Database", "N/A")  # Get IMDB rating value
                rotten_tomatoes_value = ratings_dict.get("Rotten Tomatoes", "N/A")  # Get Rotten Tomatoes value
                metacritic_value = ratings_dict.get("Metacritic", "N/A").split('/')[0]  # Get Metacritic value

                # Create a dictionary to hold all relevant movie information
                movie_data = {
                    "Title": title,  # Movie title
                    "Year": year,  # Movie year
                    "Type": movie_type,  # Movie type
                    "Released": released,  # Release date
                    "Runtime(in mins)": runtime,  # Runtime in minutes
                    "Genre": ", ".join(genres),  # Joined string of genres
                    "Director": director,  # Director of the movie
                    "Language": ", ".join(languages),  # Joined string of languages
                    "Country": ", ".join(countries),  # Joined string of countries
                    "Production": ", ".join(productions),  # Joined string of production companies
                    "Awards": awards,  # Awards won
                    "Nominations": nominations,  # Number of nominations
                    "Wins": wins,  # Number of wins
                    "IMDB Rating": imdb_rating_value,  # IMDB rating value
                    "Rotten Tomatoes": rotten_tomatoes_value,  # Rotten Tomatoes rating
                    "Metacritic": metacritic_value,  # Metacritic rating
                    "Metascore": metascore,  # Metascore value
                    "imdbVotes": imdb_votes,  # Number of votes on IMDB
                    "BoxOffice": box_office,  # Box office collection
                    "Multigenre": multigenre,  # Indicator for multiple genres
                    "Multilanguage": multilanguage,  # Indicator for multiple languages
                    "Multicompany": multicompany,  # Indicator for multiple production companies
                    "Multicountry": multicountry,  # Indicator for multiple countries
                }
                all_movies_data.append(movie_data)  # Append the movie data dictionary to the list

    # Step 15: Print message for each year after fetching
    if idx < len(years_to_fetch) - 1:  # If not the last year
        print(f"Data Extraction completed for the year {year}...", end='\r')  # Indicate year completion
    else:  # If it is the last year
        print("Data Extraction completed Successfully ! ! ! ", flush=True)  # Indicate overall completion

# Step 16: Prepare table headers for the CSV and display
headers = ["Title", "Year", "Type", "Released", "Runtime(in mins)", "Genre", "Director", "Language", "Country", "Production", "Awards", "Nominations", "Wins", "IMDB Rating", "Rotten Tomatoes", "Metacritic", "Metascore", "imdbVotes", "BoxOffice", "Multigenre", "Multilanguage", "Multicompany", "Multicountry"]

# Step 17: Create a CSV file to store the movie data
output_file = "formatted_API_data.csv"  # Define the name of the output CSV file

with open(output_file, 'w', newline='', encoding='utf-8') as csvfile:  # Open the CSV file for writing
    writer = csv.DictWriter(csvfile, fieldnames=headers)  # Create a CSV writer object with the defined headers
    writer.writeheader()  # Write the headers to the CSV file
    writer.writerows(all_movies_data)  # Write all movie data to the CSV file

# Step 18: Display Sample response in original format
display(HTML("<br><strong><h3>Sample API Data(unformatted):</h3> For 1 movie:</br></strong>"))  # Display a heading for clarity

# Define the movie title for which to fetch details
movie_title = "Gatchaman The Movie"  # Specify the movie title
url = f"http://www.omdbapi.com/?apikey={api_key}&t={movie_title}"  # Construct the URL for fetching a specific movie
response = requests.get(url)  # Make the GET request for the specific movie

# Check if the request was successful
if response.status_code == 200:  # If successful
    movie_data = response.json()  # Parse the JSON response
    print(movie_data)  # Print the JSON response to the console

# Step 19: Display formatted API data
display(HTML("<br><strong><h3>Formatted API Data:</h3></br></strong>"))  # Display a heading for clarity

# Convert to HTML table for display in the notebook
rows_for_display = [[movie[header] for header in headers] for movie in all_movies_data[:5]]  # Get first 5 entries for display

# Create an HTML table from the movie data using tabulate
html_table_display = tabulate(rows_for_display, headers=headers, tablefmt="html")

# Display the formatted table in the notebook
display(HTML(html_table_display))  # Render the HTML table in the notebook
